<a href="https://colab.research.google.com/github/TheAzouz/Project_ADA/blob/main/ML_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount ('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd "drive/My Drive/Colab Notebooks/ADA_rami"

/content/drive/My Drive/Colab Notebooks/ADA_rami


In [3]:
import csv
import pandas as pd
import numpy as np



In [4]:
ls

training_dataset.csv      training_test_set_anonymized.csv
training_dataset.csv.zip


In [5]:
pd.read_csv('training_test_set_anonymized.csv',sep=',').shape

(78812, 13)

In [6]:
df=pd.read_csv("training_dataset.csv",sep=',')
df.head()

,Unnamed: 0,user,cluster_id,CR,MR,EDR,EIDR,PR,RPR,Is_home
0,0,19,-1,0.756158,0.071661,0.583062,0.570033,0.821211,0.801806,True
1,1,19,0,0.056650,0.000000,0.695652,0.695652,0.022950,0.037273,False
2,2,19,1,0.059113,0.000000,0.750000,0.750000,0.026504,0.026315,False
3,3,19,2,0.036946,0.000000,0.333333,0.333333,0.024816,0.024017,False
4,4,19,3,0.032020,0.000000,0.615385,0.615385,0.024730,0.026114,False


In [7]:
df.shape

(180022, 10)

In [8]:
"""df_tmp=df[['CR','MR','EDR','EIDR','PR','RPR']]
df_tmp.head()"""

"df_tmp=df[['CR','MR','EDR','EIDR','PR','RPR']]\ndf_tmp.head()"

In [9]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [10]:
X=df[['CR','MR','EDR','EIDR','PR','RPR']].values
Y=df['Is_home'].values

X_train, X_test, y_train, y_test = train_test_split(preprocessing.scale(X), Y, test_size=0.2, random_state=0)

In [11]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score,precision_score,recall_score

In [ ]:
clf = make_pipeline(StandardScaler(), SVC(kernel='poly',degree=3,verbose=True,class_weight='balanced'))
clf.fit(X_train, y_train)

[LibSVM]

In [ ]:
y_pred=clf.predict(X_test)

In [19]:
f1_score(y_test,y_pred)

0.4697252269053836

In [20]:
precision_score(y_test,y_pred)

0.7821946169772257

In [21]:
recall_score(y_test,y_pred)

0.3356432125088841

In [22]:
df.head()

,Unnamed: 0,user,cluster_id,CR,MR,EDR,EIDR,PR,RPR,Is_home
0,0,19,-1,0.756158,0.071661,0.583062,0.570033,0.821211,0.801806,True
1,1,19,0,0.056650,0.000000,0.695652,0.695652,0.022950,0.037273,False
2,2,19,1,0.059113,0.000000,0.750000,0.750000,0.026504,0.026315,False
3,3,19,2,0.036946,0.000000,0.333333,0.333333,0.024816,0.024017,False
4,4,19,3,0.032020,0.000000,0.615385,0.615385,0.024730,0.026114,False


In [12]:
from keras import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,Input,LeakyReLU,ReLU
from keras.optimizers import SGD
from keras.optimizers import *
from keras import backend as K

In [13]:
from keras import backend as K

#compute recall
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

#compute precision
def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

#compute F1-score
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [16]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [17]:
input=Input((X_train.shape[1]))
d1=Dense(64)(input)
d1=ReLU()(d1)
#d1=Dropout(0.5)(d1)

d2=Dense(32)(d1)
d2=ReLU()(d2)
#d2=Dropout(0.5)(d2)

d3=Dense(16)(d2)
d3=ReLU()(d3)
#d3=Dropout(0.5)(d3)

d4=Dense(8)(d3)
d4=ReLU()(d4)
#d4=Dropout(0.5)(d4)

d5=Dense(8)(d4)
d5=ReLU()(d5)
#d5=Dropout(0.5)(d5)

d6=Dense(1, activation='sigmoid')(d5)

model=Model(inputs=input,outputs=d6)

aa=np.asarray([[0,1 ] if y_train[i] else [1,0] for i in range (len(y_train))])
aa=aa.reshape(len(y_train),2)

In [18]:
model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=f1_m)

In [19]:
model.fit(X_train,y_train,epochs=100,batch_size=10)

Epoch 1/100
14402/14402 [==============================] - 41s 3ms/step - loss: 0.2793 - f1_m: 0.4068
Epoch 2/100
14402/14402 [==============================] - 40s 3ms/step - loss: 0.2742 - f1_m: 0.4172
Epoch 3/100
14402/14402 [==============================] - 40s 3ms/step - loss: 0.2732 - f1_m: 0.4141
Epoch 4/100
14402/14402 [==============================] - 39s 3ms/step - loss: 0.2725 - f1_m: 0.4192
Epoch 5/100
14402/14402 [==============================] - 38s 3ms/step - loss: 0.2721 - f1_m: 0.4214
Epoch 6/100
14402/14402 [==============================] - 38s 3ms/step - loss: 0.2716 - f1_m: 0.4221
Epoch 7/100
14402/14402 [==============================] - 37s 3ms/step - loss: 0.2714 - f1_m: 0.4224
Epoch 8/100
14402/14402 [==============================] - 37s 3ms/step - loss: 0.2710 - f1_m: 0.4248
Epoch 9/100
14402/14402 [==============================] - 37s 3ms/step - loss: 0.2709 - f1_m: 0.4260
Epoch 10/100
14402/14402 [==============================] - 37s 3ms/step - loss: 0

KeyboardInterrupt: ignored

In [29]:
y_pred_NN=model.predict(X_test)
y_pred_NN=np.asarray([1 if y_pred_NN[i,1]>0.5 else 0 for i in range (y_pred_NN.shape[0])])

In [30]:
f1_score(y_test,y_pred_NN)

0.0

(15763,)